In [39]:
import requests
import time
import re
import os
import uuid
from bs4 import BeautifulSoup
import json
import pandas as pd
from tabulate import tabulate

### run the code below just once, takes several minutes to go through all 102 pages

In [2]:
urls = []
u_urls = set()
unique_urls = set()

#funciton to make folders for each page of the website
def make_folder(i):
    os.makedirs(str(i), exist_ok=True)
    return

#function for saving the html file of each restaurant
def html_downloader(i, u_url):
    
    response = requests.get(u_url)
        # Use uuid to generate a unique filename
    unique_filename = f"{uuid.uuid4()}.html"
    
    filepath = os.path.join(original_directory + "/data" + "/" + str(i), unique_filename)
    
        # Write the HTML content to a file
    with open(filepath, "w", encoding="utf-8") as file:
        file.write(response.text)
    return

# for managing directories of the saved files of each page
original_directory = os.getcwd()
os.makedirs('data', exist_ok=True)
os.chdir(original_directory + "/data")

#iterate through pages to save data
for i in range (1, 103): 
    
    make_folder(i)
    
    current_url = "https://guide.michelin.com/en/it/restaurants/page/" + str(i) 
    page_text = (requests.get(current_url)).text
    adress_pattern = r'href="/en/.+?/.+?/restaurant/.+?"'
    matches = re.findall(adress_pattern, page_text)
    
    for match in matches:
        urls.append(match)
        
    for url in urls:
        u_urls.add("https://guide.michelin.com" + url[6:(-1)])
        
    for u_url in u_urls:
        html_downloader(i, u_url)
        unique_urls.add(u_url)
        
    urls = []
    u_urls.clear()
    
    print(i)
    time.sleep(0.5)

os.chdir(original_directory)
    
with open('resturants_urls.txt', 'w') as file:
    for element in unique_urls:
        file.write(f"{element}\n")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102


In [46]:
print(len(unique_urls))

2037


In [11]:
print(original_directory)

/Users/amrhsnd/Desktop/ADM/Homeworks/HW3


In [43]:
def extract_info_from_html(file_path, index):
    """Extracts the required information from a single HTML file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'lxml')  # or use 'html.parser'
        
        # Prepare a dictionary to hold the extracted information
        extracted_info = {
            "index": index,
            "restaurantName": "",
            "address": "",
            "city": "",
            "postalCode": "",
            "country": "",
            "priceRange": "",
            "cuisineType": "",
            "description": "",
            "creditCards": "",
            "facilitiesServices": "",
            "phoneNumber": "",
            "website": "",
        }

        # Extract JSON-LD data
        json_ld_script = soup.find('script', type='application/ld+json')
        if json_ld_script:
            json_data = json.loads(json_ld_script.string)
            if isinstance(json_data, list):
                json_data = json_data[0]  # Get the first item if it's a list

            # Extract required fields
            extracted_info["address"] = json_data.get("address", {}).get("streetAddress", "")
            extracted_info["city"] = json_data.get("address", {}).get("addressLocality", "")
            extracted_info["postalCode"] = json_data.get("address", {}).get("postalCode", "")
            extracted_info["country"] = json_data.get("address", {}).get("addressCountry", "")
            extracted_info["restaurantName"] = json_data.get("name", "")
            extracted_info["description"] = json_data.get("description", "")
            extracted_info["phoneNumber"] = json_data.get("telephone", "")
            extracted_info["cuisineType"] = json_data.get("servesCuisine", "")
            extracted_info["website"] = json_data.get("url", "")
            extracted_info["creditCards"] = json_data.get("paymentAccepted", "")
        
        # Extract price range
        price_range_div = soup.find('div', class_='data-sheet__block--text')
        if price_range_div:
            price_text = price_range_div.get_text(strip=True)
            # Use regex to match € symbols
            match = re.search(r'[€]{1,5}', price_text)
            if match:
                extracted_info["priceRange"] = match.group(0)

        # Extract facilities and services
        services_div = soup.find('div', class_='restaurant-details__services')
        if services_div:
            extracted_info["facilitiesServices"] = services_div.get_text(strip=True)

        return extracted_info

def process_directory(directory):
    """Process all HTML files in the specified directory and extract information."""
    extracted_data_set = set()  # Using a set to hold unique dictionaries
    for index, filename in enumerate(os.listdir(directory), start=1):
        if filename.endswith('.html'):
            file_path = os.path.join(directory, filename)
            print(f"Extracting from: {file_path}")
            info = extract_info_from_html(file_path, index)
            extracted_data_set.add(frozenset(info.items()))  # Store as a frozenset for uniqueness

    return extracted_data_set

# Specify the directory containing your HTML files
html_files_directory = original_directory + "/data/1" # Change this to your actual directory
extracted_data = process_directory(html_files_directory)

# Convert the extracted data to a DataFrame
df = pd.DataFrame(extracted_data)

# Display the DataFrame as a formatted table using tabulate
print(df)

Extracting from: /Users/amrhsnd/Desktop/ADM/Homeworks/HW3/data/1/277cc236-e7db-4db3-9624-5e4c62bd7ebc.html
Extracting from: /Users/amrhsnd/Desktop/ADM/Homeworks/HW3/data/1/8accdd0e-b7ab-40f6-bfb0-3490979e56e2.html
Extracting from: /Users/amrhsnd/Desktop/ADM/Homeworks/HW3/data/1/aba34501-6937-4841-9ef4-acdd3b3f835b.html
Extracting from: /Users/amrhsnd/Desktop/ADM/Homeworks/HW3/data/1/cbb65059-10a0-4af1-a960-fb6a460ba37e.html
Extracting from: /Users/amrhsnd/Desktop/ADM/Homeworks/HW3/data/1/5c29129a-6a03-4e21-a25c-4dc8a272553d.html
Extracting from: /Users/amrhsnd/Desktop/ADM/Homeworks/HW3/data/1/7d681b4d-2558-4859-9b0d-f3850aee132b.html
Extracting from: /Users/amrhsnd/Desktop/ADM/Homeworks/HW3/data/1/6da3bd30-3ce2-4d4d-825c-81f28d7db2e8.html
Extracting from: /Users/amrhsnd/Desktop/ADM/Homeworks/HW3/data/1/40246760-a3d3-4214-80e8-3d24ec1e5d79.html
Extracting from: /Users/amrhsnd/Desktop/ADM/Homeworks/HW3/data/1/2df76603-f00c-4be5-af12-e2accef22635.html
Extracting from: /Users/amrhsnd/Deskt